# Network Centrality QCD Analysis

Import relevant packages for analysis

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as sp
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sb
import pickle

def distance(x1, y1, x2, y2):
    return ((x2-x1)**2+(y2-y1)**2)**(1/2)

This code has been tested and works with the following functions in networkx:<br>
nx.degree_centrality,nx.closeness_centrality,nx.betweenness_centrality,nx.katz_centrality,nx.pagerank,nx.eccentricity<br>
but it would also work with any other centrality measure in networkx that takes a weight as a parameter.<br>
If you instead pass a float as the function, you can find the QCD of the number of nodes within that distance on the graph.<br>
You can also pass the string "Edge Centrality" as the function to find the QCD of the edge weights.

In [1]:
#measureNames = ["Strength","Closeness Centrality","Betweenness Centrality","Katz Centrality","PageRank","Eccentricity","Edge Centrality",0.10]
#measureFuncs = [nx.degree_centrality,nx.closeness_centrality,nx.betweenness_centrality,nx.katz_centrality,nx.pagerank,nx.eccentricity,"Edge Centrality","Nodes within distance 0.10"]
measureName = "Strength"
measureFunc = nx.degree_centrality
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
colors += [colors[0],colors[7],colors[8]]
sb.set_theme()
netDict = {0:'HIP', 1:'Poisson', 2:'RSA', 3:'Stealthy0.20', 4:'Stealthy0.40',5:'Stealthy0.49', 6: 'Triangular Lattice',7:"Gaussian"}
netNameDict = {0:'HIP', 1:'Poisson', 2:'RSA', 3:'Stealthy0.20', 4:'Stealthy0.40',5:'Stealthy0.49', 6: 'Triangular Lattice',8:"Erdos-Renyi",7:"Non-stealthy Hyperuniform"}
netHyperuniform = {0:False, 1:False, 2:False, 3:True, 4:True,5:True, 6:True, 7:True}

for netNum in [0,1,2,3,4,5,7]:
    print(netDict[netNum])
    if(netNum == 1):
        numTopos = 6
    else:
        numTopos = 10

    vals = []
    for topoNum in range(numTopos):
        if(netNum == 7 and topoNum == 4):
            continue
        print(topoNum)
        num = "%02d"%(topoNum+1)
        G = nx.read_edgelist("Networks/"+netDict[netNum]+'_'+num+'_delaunay_edgelist.txt', nodetype = int, data = [('distance',float)])
        if(netNum > 2):
            for edge in G.edges():
                G.edges()[edge]['distance']/=100
        for edge in G.edges():
            G.edges()[edge]['strength'] = 1/G.edges()[edge]['distance']
                
        G = G.subgraph(max(list(nx.connected_components(G)),key = len))
        #Centrality Analysis
        if(callable(measureFunc)):
            if(measureFunc == nx.degree_centrality):  
                vals += [G.degree(n,weight='strength') for n in G.nodes()]
            elif(measureFunc == nx.closeness_centrality):
                vals += list(measureFunc(G, distance = 'distance').values())
            elif(measureFunc == nx.eccentricity):
                distDict = dict(nx.all_pairs_dijkstra_path_length(G, weight = 'distance'))
                vals += list(measureFunc(G,sp=distDict).values())
            else:
                vals += list(measureFunc(G, weight = 'distance').values())
        elif(type(measureFunc) == float):
                dist = measureFunc
                pct = 10
                for node in nodes:
                    G2 = nx.ego_graph(G,node,radius=dist,distance='distance')
                    vals.append(len(G2))
        elif(measureFunc == "Edge Centrality"):
            vals += [G.edges()[E]['distance'] for E in G.edges()]

    resDF = pd.DataFrame(columns=["Mean","Std","Var","CV","Skew","Kurt","Med","IQR","QCD"])
    for R in geometricConnectionRs:
        measAvg = np.average(measureDict[R])
        measStd = np.std(measureDict[R])
        measVar = np.var(measureDict[R])
        measCV = measStd/measAvg
        measSkew = sp.skew(measureDict[R])
        measKurt = sp.kurtosis(measureDict[R])
        measMed = np.median(measureDict[R])
        measIQR = np.percentile(measureDict[R],75)-np.percentile(measureDict[R],25)
        measQCD = measIQR/(np.percentile(measureDict[R],75)+np.percentile(measureDict[R],25))
        resDF.loc[R] = [measAvg,measStd,measVar,measCV,measSkew,measKurt,measMed,measIQR,measQCD]

NameError: name 'nx' is not defined